In [1]:
# Link
# https://towardsdatascience.com/data-101s-spatial-visualizations-and-analysis-in-python-with-folium-39730da2adf

In [3]:
import numpy as np
import pandas as pd
import folium

from math import radians, cos, sin, asin, sqrt
from folium.plugins import HeatMap

# Data load and preparation

In [49]:
df = pd.read_csv('test.csv')
df['datetime'] = pd.to_datetime(df['pickup_datetime'], format='%Y-%m-%d %H:%M:%S UTC')
df['date'] = df['datetime'].dt.date
df['hour'] = df['datetime'].dt.round('H').dt.hour
#df = df.loc[df['date'].astype(str) == '2011-12-13']

data = df[['hour', 'datetime', 'pickup_latitude', 'pickup_longitude', 'passenger_count']]
data.columns = ['Hour', 'Datetime', 'Latitude', 'Longitude', 'Active_Users']

data.head(10)

,Hour,Datetime,Latitude,Longitude,Active_Users
0,13,2015-01-27 13:08:24,40.763805,-73.973320,1
1,13,2015-01-27 13:08:24,40.719383,-73.986862,1
2,12,2011-10-08 11:53:44,40.751260,-73.982524,1
3,21,2012-12-01 21:12:12,40.767807,-73.981160,1
4,21,2012-12-01 21:12:12,40.789775,-73.966046,1
5,21,2012-12-01 21:12:12,40.765547,-73.960983,1
6,12,2011-10-06 12:10:20,40.773204,-73.949013,1
7,12,2011-10-06 12:10:20,40.646636,-73.777282,1
8,12,2011-10-06 12:10:20,40.709638,-74.014099,1
9,15,2014-02-18 15:22:20,40.765519,-73.969582,1


In [50]:
banners = pd.read_csv('banners.csv')
banners['Latitude'] = banners['Latitude'].astype(float)
banners['Longitude'] = banners['Longitude'].astype(float)

In [51]:
banners.City.value_counts()

New-York    7
Name: City, dtype: int64

In [52]:
banners

,City,Type,Latitude,Longitude,Address
0,New-York,Megaboard,40.763805,-73.973320,Manhattan
1,New-York,Billboard,40.719383,-73.986862,China Town
2,New-York,Indoor,40.751260,-73.982524,Broadway
3,New-York,Lightbox,40.767807,-73.981160,College Station
4,New-York,Mediaboard,40.789775,-73.966046,Manhattan
5,New-York,Busstop,40.765547,-73.960983,Manhattan
6,New-York,Underground,40.773204,-73.949013,Manhattan


# Rankings

In [53]:
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great icrcle distance between two points
    on the earth (specified in decimal degrees)
    """
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371
    return c * r

In [54]:
def count_active_users(bs_data, radius, lat, lon):
    """
    Calculate the number of active users at specified location,
    iterating over all base stations
    """
    users = 0
    for bs_row in range(bs_data.shape[0]):
        bs = bs_data.iloc[bs_row]
        lat1 = bs[2]
        lon1 = bs[3]
        lat2 = lat
        lon2 = lon
        distance = haversine(lat1, lon1, lat2, lon2)
        if distance <= radius and not np.isnan(bs[4]):
            users += bs[4]
    return users

In [55]:
%%time
rankings = banners.copy()
rankings['users'] = 0

for banner_row in range(rankings.shape[0]):
    score = 0
    banner = rankings.iloc[banner_row]
    users = count_active_users(data, 1.00, banner[2], banner[3])
    rankings.iloc[banner_row, 5] = float(users)
    
    if (banner_row + 1) % 10 == 0:
        print(f'# of processed banners: {banner_row + 1}')
        
rankings['users'] = round(rankings['users'])

CPU times: user 12.6 s, sys: 38.6 ms, total: 12.6 s
Wall time: 12.7 s


In [56]:
rankings.sort_values('users', ascending=False).loc[rankings.City=='New-York']

,City,Type,Latitude,Longitude,Address,users
2,New-York,Indoor,40.751260,-73.982524,Broadway,2907.0
0,New-York,Megaboard,40.763805,-73.973320,Manhattan,2318.0
3,New-York,Lightbox,40.767807,-73.981160,College Station,1780.0
5,New-York,Busstop,40.765547,-73.960983,Manhattan,1548.0
6,New-York,Underground,40.773204,-73.949013,Manhattan,978.0
1,New-York,Billboard,40.719383,-73.986862,China Town,908.0
4,New-York,Mediaboard,40.789775,-73.966046,Manhattan,494.0


In [57]:
count_active_users(data, 1.0, 40.763805, -73.973320)

2318

# Heatmap with time

In [58]:
def generateBaseMap(default_location=[40.763805, -73.973320],
                    default_zoom_start=11, tiles="Stamen Terrain"):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start,
                         tiles="Stamen Terrain")
    return base_map

In [61]:
#Normal heatmap

base_map = generateBaseMap()
HeatMap(data=data[['Latitude', 'Longitude', 'Active_Users']].groupby(['Latitude', 'Longitude']).sum().reset_index().values.tolist(),
       radius=14, max_zoom=13).add_to(base_map)
base_map

In [62]:
df_hour_list = []
for hour in data.Hour.sort_values().unique():
    df_hour_list.append(data.loc[data.Hour == hour, ['Latitude', 'Longitude', 'Active_Users']]. \
                        groupby(['Latitude', 'Longitude']).sum().reset_index().values.tolist())

In [63]:
new_york = [40.763805, -73.973320]

In [65]:
#Heatmap with time

from folium.plugins import HeatMapWithTime
map_ = generateBaseMap(default_location=new_york, default_zoom_start=11.5)
HeatMapWithTime(df_hour_list, radius=35, gradient={0.2: 'blue', 0.35: 'lime', 0.5: 'orange', 0.65: 'red'},
               min_opacity=0.5, max_opacity=1, use_local_extrema=True).add_to(map_)

for index, row in rankings.iterrows():
    if row[1] == 'Megaboard':
        icon = 'film'
    elif row[1] == 'Busstop':
        icon = 'plane'
    elif row[1] == 'LCD screen':
        icon = 'phone'
    elif row[1] == 'Mediaboard':
        icon = 'unchecked'
    elif row[1] == 'Lightbox':
        icon = 'inbox'
    elif row[1] == 'Indoor':
        icon = 'home'
    else:
        icon = 'info-sign'
    
    if row[5] >= 0 and row[5] < 10000:
        color = 'lightblue'
    elif row[5] >= 10000 and row[5] < 20000:
        color = 'darkblue'
    elif row[5] >= 20000 and row[5] < 30000:
        color = 'lightgreen'
    elif row[5] >= 30000 and row[5] < 40000:
        color = 'orange'
    elif row[5] >= 40000:
        color = 'red'
    
    folium.Marker([row[2], row[3]],
              popup=row[1] + str('\n') + row[4] + str('\n') + str(row[5]),
              icon=folium.Icon(color=color, icon=icon) 
             ).add_to(map_)

import branca
colormap = branca.colormap.LinearColormap(['lightblue', 'darkblue', 'lightgreen', 'orange', 'red'])  #YlOrRd_09.scale(0, 8500)
colormap = colormap.to_step(index=[0, 10000, 20000, 30000, 40000, 50000])
colormap.caption = 'Daily active subscribers exposed to banner (28 Nov 2019)'
colormap.add_to(map_)

#folium.ClickForMarker().add_to(map_)
folium.LatLngPopup().add_to(map_)

map_

In [67]:
map_.save('results/banners_map.html')